<a href="https://colab.research.google.com/github/ANKITPODDER2000/CNN_TF/blob/master/tensorflowhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q -U tf-hub-nightly
#!pip install -q tfds-nightly

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow_hub as hub
import cv2

In [ ]:
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" 

In [ ]:
image_shape = (224,224,3)

model1 = tf.keras.models.Sequential([
    hub.KerasLayer(classifier_url,input_shape=image_shape)
])

In [ ]:
import numpy as np
import PIL.Image as image
image_url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg'
img = tf.keras.utils.get_file("img.jpg",image_url)
img = image.open(img).resize((224,224))
img

In [ ]:
img = np.array(img)/255.0
print("Shape of img := ",img.shape)

In [ ]:
result = model1.predict(img[np.newaxis])
result.shape

In [ ]:
result.argmax()

In [ ]:
label_path = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"
label = tf.keras.utils.get_file("label.txt",label_path)
label = open(label).read().split("\n")

In [ ]:
print("Predicted Class : ",label[result.argmax()])

In [ ]:
image_url = "https://4.imimg.com/data4/IE/UJ/MY-21747848/men-s-casual-blazer-500x500.jpg"
img_me = tf.keras.utils.get_file("myimg3.jpg",image_url)

img_me = cv2.imread(img_me)
img_me = cv2.resize(img_me,(224,224))
img_me = img_me / 255.0
plt.imshow(img_me)
print("Shape of image := ",img_me.shape)
predict = model1.predict(img_me[np.newaxis])
print("Predicted Class := ",label[predict.argmax()])
plt.title("Prediction : "+label[predict.argmax()])

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

In [ ]:
train_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=(1.0 / 255.0))

In [ ]:
iamge_data = train_data.flow_from_directory(data_root,target_size=(224,224),batch_size=32)

In [ ]:
iamge_data.n

In [ ]:
iamge_data.class_indices

In [ ]:
test_image = iamge_data[0][0]
test_label = iamge_data[0][1]
print("test image shape := ",test_image.shape)
print("test label shape := ",test_label.shape)

In [ ]:
prediction = model1.predict(test_image)

In [ ]:
prediction = np.argmax(prediction, axis=-1)
print("Sahpe of prediction := ",prediction.shape)
lbl = np.array(label)
print(lbl[prediction])

In [ ]:
plt.imshow(test_image[0])

In [ ]:
headless_classifier = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"

In [ ]:
classifier_input = hub.KerasLayer(headless_classifier,input_shape=image_shape)

In [ ]:
classifier_input.trainable = False

In [ ]:
model2 = tf.keras.models.Sequential([
    classifier_input,
    tf.keras.layers.Dense(5,activation="softmax")
])
model2.summary()

In [ ]:
model2.compile(optimizer="adam",loss=tf.keras.losses.categorical_crossentropy,metrics=['acc'])

In [ ]:
history2 = model2.fit(iamge_data,epochs=10,steps_per_epoch=iamge_data.n//iamge_data.batch_size)

In [ ]:
model2.predict_classes(iamge_data[0][0][0:2])

In [ ]:
iamge_data[0][1][0:2]

In [ ]:
import time
t = time.time()

In [ ]:
path = "/content/drive/My Drive/CNN/fruit"
model2.save(path,save_format='tf')

In [ ]:
reloaded = tf.keras.models.load_model(path)

In [ ]:
reloaded.predict_classes(iamge_data[0][0])